In [ ]:
from typing import List, Dict, Tuple
import json
import xml.etree.ElementTree as ET

from sympy import S, Expr, N
from sympy.geometry import intersection, Point, Ray, Circle
from colormath.color_objects import sRGBColor

from IPython.display import HTML

In [ ]:
RGB = Dict[str, float] # {r: float, g: float, b: float}
Skin = Tuple[str, RGB] # (title, RGB)

PathComponent = Tuple[str, List[float]] # (command, params)

# Construct the 'd' attribute
class Path:
    def __init__(self):
        self.components: List[PathComponent] = []
    
    def M(self, params: List[float]) -> 'Path':
        self.components.append(('M', params))
        return self
    
    def L(self, params: List[float]) -> 'Path':
        self.components.append(('L', params))
        return self
    
    def A(self, params: List[float]) -> 'Path':
        self.components.append(('A', params))
        return self
    
    def Z(self) -> 'Path':
        self.components.append(('Z', []))
        return self
    
    def stringify(self) -> str:
        def stringify_component(component: PathComponent) -> str:
            if len(component[1]) == 0:
                return component[0]
            else:
                return f"{component[0]} {' '.join([str(param) for param in component[1]])}"
        return ''.join([stringify_component(component) for component in self.components])

def SVGElement(tag: str, attributes: Dict[str, str]):
    element = ET.Element(tag)
    for (key, value) in attributes.items():
        element.set(key, str(value))
    return element

class SVG:
    def __init__(self, width: float, height: float):
        self.root = SVGElement('svg', {
            'xmlns': 'http://www.w3.org/2000/svg',
            'xmlns:xlink': 'http://www.w3.org/1999/xlink',
            'width': width,
            'height': height,
            'viewBox': f'0 0 {width} {height}',
        })
        
        self.tree = ET.ElementTree(self.root)
    
    def add_def(self, id: str, tag: str, attributes: Dict[str, str]):
        defs_container = self.root.find('defs')
        if defs_container is None:
            defs_container = ET.SubElement(self.root, 'defs')
        def_element = SVGElement(tag, {
            'id': id,
            **attributes,
        })
        defs_container.append(def_element)
    
    def add_use(self, def_id: str, attributes: Dict[str, str]):
        use_element = SVGElement('use', {
            'xlink:href': '#' + def_id,
            **attributes,
        })
        self.root.append(use_element)

    def add_comment(self, text: str):
        self.root.append(ET.Comment(text))
    
    def output(self, filename):
        self.tree.write(filename, encoding='utf-8')

In [ ]:
decimal_place_accuracy = 3

palette_segment_count = 6

canvas_size = 100
inner_radius = 30
outer_radius = 45
divider_width = 5

In [ ]:
def import_picked_skins(title: str) -> List[RGB]:
    with open(f'../output/{title}.json') as f:
        items = json.loads(f.read())
        skins = [(item['title'], {
            'r': item['r'],
            'g': item['g'],
            'b': item['b'],
        }) for item in items]
    return skins

In [ ]:
def origin_to_top_left(point: Point, canvas_width: Expr, canvas_height: Expr) -> Point:
    flipped = Point(point.x, - point.y)
    return flipped.translate(canvas_width / 2, canvas_height / 2)

def point_to_svg_params(point: Point, canvas_width: Expr, canvas_height: Expr) -> List[float]:
    shifted_point = origin_to_top_left(point, canvas_width, canvas_height)
    return [
        round(N(shifted_point.x), decimal_place_accuracy),
        round(N(shifted_point.y), decimal_place_accuracy),
    ]

# Returns a path that represents a single segment of a palette,
# which can be reused with different angles of rotation
# Angle 0 is the vertical line going upwards
def create_single_palette_segment(start_angle: Expr, angle_per_segment: Expr, canvas_size: float, inner_radius: float, outer_radius: float, divider_width: float) -> Path:
    # Circles
    inner_circle = Circle(Point(0, 0), inner_radius)
    outer_circle = Circle(Point(0, 0), outer_radius)
    
    # Lines
    vertical_ray = Ray(Point(0, 0), angle=S.Pi/2) # A ray from the origin going upwards
    rays_around_vertical_ray = [
        vertical_ray.translate(- divider_width / 2, 0), # Left side of the vertical line
        vertical_ray.translate(+ divider_width / 2, 0), # Right side of the vertical line
    ]
    enclosing_rays = [
        rays_around_vertical_ray[0].rotate(start_angle, Point(0, 0)),
        rays_around_vertical_ray[1].rotate(start_angle + angle_per_segment, Point(0, 0)),
    ]
    
    # Intersections
    inner_intersections = [
        intersection(inner_circle, enclosing_rays[0])[0],
        intersection(inner_circle, enclosing_rays[1])[0],
    ]
    outer_intersections = [
        intersection(outer_circle, enclosing_rays[0])[0],
        intersection(outer_circle, enclosing_rays[1])[0],
    ]
    
    return Path().M(
        point_to_svg_params(inner_intersections[0], canvas_size, canvas_size)
    ).A(
        [
            inner_radius, inner_radius,
            0, 0, 0,
            *point_to_svg_params(inner_intersections[1], canvas_size, canvas_size)
        ]
    ).L(
        point_to_svg_params(outer_intersections[1], canvas_size, canvas_size)
    ).A(
        [
            outer_radius, outer_radius,
            0, 0, 1,
            *point_to_svg_params(outer_intersections[0], canvas_size, canvas_size)
        ]
    ).Z()

def create_svg(palette_segment_count: int, canvas_size: float, inner_radius: float, outer_radius: float, divider_width: float, skins: List[Skin]) -> SVG:
    svg = SVG(canvas_size, canvas_size)
    
    single_palette_segment_path = create_single_palette_segment(
        0,
        S.Pi * 2 / palette_segment_count,
        canvas_size,
        inner_radius,
        outer_radius,
        divider_width,
    )
    
    svg.add_def('seg', 'path', {
        'd': single_palette_segment_path.stringify()
    })
    
    for i in range(palette_segment_count):
        svg.add_comment(skins[i][0])
        rotation_angle = 360 * i / palette_segment_count
        rgb = sRGBColor(skins[i][1]['r'], skins[i][1]['g'], skins[i][1]['b'], is_upscaled=True)
        svg.add_use('seg', {
            'transform': f'rotate({rotation_angle} {canvas_size / 2} {canvas_size / 2})',
            'fill': rgb.get_rgb_hex(),
        })
        
    return svg

In [ ]:
create_svg(
    palette_segment_count,
    canvas_size,
    inner_radius,
    outer_radius,
    divider_width,
    import_picked_skins('average-skins'),
).output('../output/average-skins-icon.svg')

create_svg(
    palette_segment_count,
    canvas_size,
    inner_radius,
    outer_radius,
    divider_width,
    import_picked_skins('neutral-skins'),
).output('../output/neutral-skins-icon.svg')

create_svg(
    palette_segment_count,
    canvas_size,
    inner_radius,
    outer_radius,
    divider_width,
    import_picked_skins('distinct-skins'),
).output('../output/distinct-skins-icon.svg')